# KoBERT 기반 감성 분류

이 노트북은 KoBERT 모델을 이용해 리뷰 데이터를 학습하고 감성 분류 정확도를 평가하는 예제입니다.

In [ ]:
# 0. 필요 라이브러리 설치
!pip install datasets transformers scikit-learn torch sentencepiece

In [ ]:
# 1. 라이브러리 임포트
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score

## 데이터 준비

In [ ]:
# 2. KoBERT 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
print("KoBERT 토크나이저 로드 완료")

In [ ]:
# 3. CSV 파일 → Hugging Face Dataset 로드
dataset = load_dataset("csv", data_files={"train": "review.csv"})
print("CSV 로드 완료")
print("데이터 컬럼:", dataset["train"].column_names)

In [ ]:
# 4. 토크나이저 적용 함수
def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True, max_length=128, return_token_type_ids=False)

In [ ]:
# 5. 전체 데이터셋에 토큰화 적용
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["review"])
tokenized_dataset.set_format("torch")
print("토크나이징 완료")

In [ ]:
# 6. Train-Test Split (80% 훈련, 20% 테스트)
split_datasets = tokenized_dataset["train"].train_test_split(test_size=0.2)
tokenized_dataset = split_datasets
print("Train/Test 분할 완료")
print("Train 데이터 개수:", len(tokenized_dataset["train"]))
print("Test 데이터 개수:", len(tokenized_dataset["test"]))

## 모델 준비

In [ ]:
# 7. KoBERT 모델 로드
model = AutoModelForSequenceClassification.from_pretrained("skt/kobert-base-v1", num_labels=7)
print("KoBERT 모델 로드 완료")

In [ ]:
# 라벨을 int 형식으로 변환
def cast_to_int(example):
    example["label"] = int(example["label"])
    return example

tokenized_dataset = tokenized_dataset.map(cast_to_int)

## 학습 설정

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_total_limit=2,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    logging_dir='./logs',
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

## 학습 및 평가

In [ ]:
print("학습 시작")
trainer.train()
print("학습 완료")

In [ ]:
print("테스트 세트 평가")
predictions = trainer.predict(tokenized_dataset["test"])
preds = torch.argmax(torch.tensor(predictions.predictions), dim=1)
labels = tokenized_dataset["test"]["label"][:]
acc = accuracy_score(labels, preds.tolist())
print(f"최종 정확도: {acc:.4f}")